In [15]:
import numpy as np
from torch.utils.serialization import load_lua


# load Completion Network
model_path = './completionnet_places2.t7'
data = load_lua(model_path)
model = data.model
model.evaluate()
datamean = data.mean
print (datamean)

tensor([ 0.4560,  0.4472,  0.4155])


In [11]:
i = 48
value = model.modules[i].weight.numpy()
print (i, model.modules[i], np.max(value), np.min(value), np.mean(value))

48 nn.SpatialConvolution(32 -> 3, 3x3, 1, 1, 1, 1) 1.2813634 -1.2358687 -0.0033915918


In [13]:
print (model.modules[0].weight.numpy().shape)
#print (model.modules)
for i in range(len(model.modules)):
    #value = model.modules[i].weight.numpy()
    print (i, model.modules[i])#, np.max(value), np.min(value), np.mean(value))
# model.modules[i].weight.numpy()
#print (model.modules[1].weight, model.modules[1].bias, model.modules[1].running_var, model.modules[1].running_mean)

(64, 4, 5, 5)
0 nn.SpatialConvolution(4 -> 64, 5x5, 1, 1, 2, 2)
1 nn.SpatialBatchNormalization
2 nn.ReLU
3 nn.SpatialConvolution(64 -> 128, 3x3, 2, 2, 1, 1)
4 nn.SpatialBatchNormalization
5 nn.ReLU
6 nn.SpatialConvolution(128 -> 128, 3x3, 1, 1, 1, 1)
7 nn.SpatialBatchNormalization
8 nn.ReLU
9 nn.SpatialConvolution(128 -> 256, 3x3, 2, 2, 1, 1)
10 nn.SpatialBatchNormalization
11 nn.ReLU
12 nn.SpatialDilatedConvolution(256 -> 256, 3x3, 1, 1, 1, 1, 1, 1)
13 nn.SpatialBatchNormalization
14 nn.ReLU
15 nn.SpatialDilatedConvolution(256 -> 256, 3x3, 1, 1, 1, 1, 1, 1)
16 nn.SpatialBatchNormalization
17 nn.ReLU
18 nn.SpatialDilatedConvolution(256 -> 256, 3x3, 1, 1, 2, 2, 2, 2)
19 nn.SpatialBatchNormalization
20 nn.ReLU
21 nn.SpatialDilatedConvolution(256 -> 256, 3x3, 1, 1, 4, 4, 4, 4)
22 nn.SpatialBatchNormalization
23 nn.ReLU
24 nn.SpatialDilatedConvolution(256 -> 256, 3x3, 1, 1, 8, 8, 8, 8)
25 nn.SpatialBatchNormalization
26 nn.ReLU
27 nn.SpatialDilatedConvolution(256 -> 256, 3x3, 1, 1, 16, 16,

In [3]:
from layer import *
import tensorflow as tf


In [5]:
is_training = tf.placeholder(tf.bool, [])
x = tf.placeholder(tf.float32, [8, 256, 256, 4])
with tf.variable_scope('generator2'):
    with tf.variable_scope('conv1'):
        x = conv_layer(x, [5, 5, 4, 64], 1, 
                       initializer=tf.constant_initializer(model.modules[0].weight.numpy()))
        x = batch_normalize(x, is_training,
                            initializer_beta=tf.constant_initializer(model.modules[1].bias.numpy()),
                            initializer_scale=tf.constant_initializer(model.modules[1].weight.numpy()), 
                            initializer_mean=tf.constant_initializer(model.modules[1].running_mean.numpy()), 
                            initializer_var=tf.constant_initializer(model.modules[1].running_var.numpy())
                           )
        x = tf.nn.relu(x)